In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
torch.cuda.empty_cache()

In [3]:
path_img_106 = []
path_img_ngt = []

In [4]:
root = r"C:\Users\lucyc\Desktop\DatSet_Activate\105_face"
dir_list = os.listdir(root)
for dir_person in dir_list:
    path_img_106.append([root+"\\"+dir_person+"\\"+x for x in os.listdir(root + "\\" + dir_person) if x[-1] == "g" or x[-1] == "G"])

root = r"C:\Users\lucyc\Desktop\DatSet_Activate\diff_face"
path_img_ngt = [root+"\\"+x for x in os.listdir(root)[:50000]]

In [5]:
path_data_trp = []

count = 0
# for cla_imgs in path_img_106:
#     for i in range(1, len(cla_imgs), 2):
#         path_data_trp.append((cla_imgs[i], cla_imgs[i-1], path_img_ngt[count]))
#         count += 1

# for cla_i in range(1, len(path_img_106), 2):
#     j = 1
#     while j < len(path_img_106[cla_i-1]) and j//2 < len(path_img_106[cla_i]):
#         path_data_trp.append((path_img_106[cla_i-1][j-1], path_img_106[cla_i-1][j], path_img_106[cla_i][j//2]))
#         j += 2
#         count += 1

# for cla_i in range(2, len(path_img_106), 2):
#     j = 1
#     while j < len(path_img_106[cla_i-1]) and j//2 < len(path_img_106[cla_i]):
#         path_data_trp.append((path_img_106[cla_i-1][j-1], path_img_106[cla_i-1][j], path_img_106[cla_i][j//2]))
#         j += 2
#         count += 1

import random

def random_pop(lst):
    if lst:  # 检查列表是否非空
        index = random.randrange(len(lst))  # 选择一个随机索引
        return lst.pop(index)  # 弹出并返回该索引对应的元素
    else:
        return None  # 如果列表为空，则返回None

def get_ngt(i):
    all_path = []
    
    def rebuild():
        nonlocal all_path
        for j, x in enumerate(path_img_106):
            if j == i:
                continue
            all_path += x
            
    rebuild()
    while True:
        select = random_pop(all_path)
        if select != None:
            yield select
        else:
            rebuild()
            yield random_pop(all_path)

for i in range(len(path_img_106)):
    getngt = get_ngt(i)
    for j in range(len(path_img_106[i])):
        for k in range(j+1,len(path_img_106[i])):
            path_data_trp.append([path_img_106[i][j], path_img_106[i][k], next(getngt)])


In [6]:
len(path_data_trp)

1515807

In [7]:
import random
path_data_trp = random.sample(path_data_trp, len(path_data_trp)//10)

In [9]:
img = Image.open(path_data_trp[0][0])

In [11]:

# 定义转换操作
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # 将PIL图像或NumPy ndarray转换为FloatTensor。
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # 标准化，使用ImageNet的均值和标准差
                         std=[0.229, 0.224, 0.225])
])

from boring import TrpDataSet

# 创建数据集
dataset = TrpDataSet(path_data_trp, transform=transform)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset)*0.9), int(len(train_dataset)*0.1)+1])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=3, pin_memory=True)


In [12]:
a = next(iter(train_loader))

In [13]:
a[0].shape

torch.Size([128, 3, 224, 224])

In [14]:
from torchvision import models
# 加载预训练的VGG-16模型
vgg16 = models.vgg16(pretrained=True)

C:\Users\lucyc\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lucyc\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
vgg16.classifier[0].in_features

25088

In [16]:
for p in vgg16.parameters():
    p.requires_grad = False

In [17]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
class SiameseCNN_VGG16(nn.Module):

    def __init__(self):
        super(SiameseCNN_VGG16, self).__init__()
        # 定义网络层
        self.vgg16 = model = nn.Sequential(
                            vgg16.features,
                            vgg16.avgpool
        )
        self.feature = vgg16.classifier[0].in_features
        # 定义全连接层
        self.dens = nn.Linear(self.feature, 1024)
        self.drop = nn.Dropout(p=0.5, inplace=False)
        self.dens2 = nn.Linear(1024, 1024)

    def forward(self, x):
        # 定义前向传播
        #x = x.flatten(start_dim=1)
        x = self.vgg16(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.dens(x))
        x = self.drop(x)
        x = F.relu(self.dens2(x))
        return x


In [20]:
class TripLoss(nn.Module):
    def __init__(self):
        super(TripLoss, self).__init__()
        # 这里可以初始化任何需要的参数或层

    def forward(self, anchor, positive, negative):
        # 自定义损失的计算逻辑
        # 例如，我们可以计算平均绝对误差     
        # 计算锚点与正负样本之间的距离
        distance_positive = (anchor - positive).pow(2).sum(1)  # 对每个元素平方后求和
        distance_negative = (anchor - negative).pow(2).sum(1)
        
        # 计算三元组损失
        losses = F.relu(distance_positive - distance_negative + 3)
        
        # 返回平均损失
        return losses.mean()

In [21]:
torch.cuda.empty_cache()
torch.autograd.set_detect_anomaly(True)
# 构建模型
model = SiameseCNN_VGG16()
print(model)

model.to(device)  # 将模型发送到GPU，如果有的话

# 定义损失函数和优化器
criterion = TripLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练模型
num_epochs = 30

for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式

    loss_acc = 0.
    train_num = 0
    for inputs in train_loader:
        anchor = inputs[0].to(device)
        positive = inputs[1].to(device)
        negative = inputs[2].to(device)
        
        # 前向传播
        anchor = model(anchor)
        positive = model(positive)
        negative = model(negative)

        optimizer.zero_grad()  # 清除之前的梯度
        loss = criterion(anchor, positive, negative)
        # 反向传播和优化
        loss.backward()  # 反向传播计算当前的梯度
        optimizer.step()  # 更新参数

        loss_acc += loss.item()
        train_num += 1

        print(loss)

    model.eval() 
    val_loss_acc = 0
    val_num = 0
    with torch.no_grad():
        for inputs in val_loader:
            anchor = inputs[0].to(device)
            positive = inputs[1].to(device)
            negative = inputs[2].to(device)
            
            anchor = model(anchor)
            positive = model(positive)
            negative = model(negative)

            loss = criterion(anchor, positive, negative)
            #loss += abs(5-criterion(anchor, negative))
            val_loss_acc += loss
            val_num += 1

    print("Epoch [{}/{}], Loss: {:.2f}, Val_loss: {:.2f}".format(epoch+1, num_epochs, loss_acc/train_num, val_loss_acc/val_num))


SiameseCNN_VGG16(
  (vgg16): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
   

KeyboardInterrupt: 

In [22]:
def loss_cal(v1, v2):
    apd = torch.sum((v1 - v2) ** 2, dim=1)
    return apd.mean() # 平均欧几里得距离

model.eval()
tl = iter(test_loader)
pass_num = 0
fail_num = 0
with torch.no_grad():
    for _ in range(250):
        anchor, positive, negative = next(tl)
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        a = model(anchor)
        p = model(positive)
        n = model(negative)
        
        if loss_cal(a,p) < loss_cal(a,n):
            pass_num += 1
        else:
            fail_num += 1
            
        print("ap: {:.2f}, an: {:.2f}".format(loss_cal(a,p),loss_cal(a,n)))

print(pass_num)
    

ap: 3.07, an: 25.15
ap: 5.09, an: 8.51
ap: 4.55, an: 22.42
ap: 5.40, an: 7.47
ap: 22.41, an: 25.40
ap: 6.15, an: 53.34
ap: 2.11, an: 18.96
ap: 2.73, an: 28.57
ap: 2.05, an: 5.41
ap: 6.13, an: 18.59
ap: 6.17, an: 30.32
ap: 5.75, an: 47.39
ap: 3.34, an: 48.06
ap: 3.03, an: 15.60
ap: 3.87, an: 22.77
ap: 3.80, an: 28.62
ap: 5.77, an: 12.40
ap: 4.04, an: 30.98
ap: 12.96, an: 58.68
ap: 2.92, an: 15.96
ap: 3.28, an: 20.64
ap: 1.72, an: 19.75
ap: 8.10, an: 39.74
ap: 2.96, an: 24.73
ap: 2.44, an: 38.57
ap: 4.64, an: 11.03
ap: 1.42, an: 17.89
ap: 4.44, an: 14.32
ap: 0.96, an: 32.24
ap: 3.46, an: 21.05
ap: 4.02, an: 34.93
ap: 4.40, an: 17.01
ap: 4.46, an: 11.85
ap: 3.98, an: 18.63
ap: 5.51, an: 50.86
ap: 1.34, an: 13.93
ap: 1.21, an: 32.53
ap: 5.22, an: 20.95
ap: 2.55, an: 7.59
ap: 1.92, an: 35.75
ap: 5.40, an: 36.07
ap: 2.54, an: 62.01
ap: 6.78, an: 18.70
ap: 7.91, an: 23.33
ap: 2.91, an: 9.04
ap: 2.89, an: 40.73
ap: 9.35, an: 19.53
ap: 4.02, an: 35.68
ap: 1.89, an: 5.83
ap: 2.46, an: 3.54
ap: 1

In [79]:
for a, b in model.named_parameters():
    print(a)
    b.requires_grad = True

vgg16.0.0.weight
vgg16.0.0.bias
vgg16.0.2.weight
vgg16.0.2.bias
vgg16.0.5.weight
vgg16.0.5.bias
vgg16.0.7.weight
vgg16.0.7.bias
vgg16.0.10.weight
vgg16.0.10.bias
vgg16.0.12.weight
vgg16.0.12.bias
vgg16.0.14.weight
vgg16.0.14.bias
vgg16.0.17.weight
vgg16.0.17.bias
vgg16.0.19.weight
vgg16.0.19.bias
vgg16.0.21.weight
vgg16.0.21.bias
vgg16.0.24.weight
vgg16.0.24.bias
vgg16.0.26.weight
vgg16.0.26.bias
vgg16.0.28.weight
vgg16.0.28.bias
dens.weight
dens.bias
dens2.weight
dens2.bias


In [44]:
for a, b in model.named_parameters():
    print(a)
    print(b.requires_grad)

conv1.weight
True
conv1.bias
True
conv2.weight
True
conv2.bias
True
conv3.weight
True
conv3.bias
True
conv4.weight
True
conv4.bias
True
conv5.weight
True
conv5.bias
True
dens.weight
True
dens.bias
True


In [25]:
torch.save(model, 'face_v2_full.pth')

In [26]:
torch.save(model.state_dict(), 'face_v2.pth')